In [596]:
from mendeleev import element
from thermo.chemical import Chemical
import sympy.physics.units as units
import scipy.constants as constants
amu = constants.physical_constants["atomic mass unit-kilogram relationship"][0]
import math
import sys
sys.path.insert(0, '../../files/ionprinter/simulation/utilities/')
import util

# Inputs

In [610]:
#GLOBALS

print_species = element('Aluminum')

print_speed = 10.0 #g/hour

#convert to kg/s, kick CGS to the curb.
print_speed /= 1000.0
print_speed /= 3600.0

number_of_bowties = 10000.0

build_platform_distance = 0.03 #m

#BOWTIE

bowtie_nozzle_radius = 0.00033#m

bowtie_hot_area = 1e-6 #m^2
graphite_emissivity = 0.95

bowtie_temperature = 1400.0 #Kelvin


#CHAMBER

ionization_chamber_radius = 0.0015 #m
ionization_chamber_length = 0.01 #m
acceleration_voltage = 100 #v
ionization_efficiency = 0.3

ICP_frequency = 1000000
ICP_peak_B = 0.05


# import pandas as pd
# data = [[1, 2], [3, 4]]
# pd.DataFrame(data, columns=["Foo", "Bar"])


In [613]:
#Total n of atoms that must be emitted to achieve desired mass deposition per hour
atoms_per_second = print_speed/(print_species.mass*amu)

moles_per_second = atoms_per_second/constants.Avogadro

#Volume deposited per hour.
hourly_print_volume = ((print_speed*3600.0)/(print_species.density*1000.0))/1.0e-6

total_beam_current = atoms_per_second*constants.elementary_charge
per_chamber_beam_current = total_beam_current/number_of_bowties

print("Atoms per second: \nper chamber %0.5g atoms/s \ntotal %0.5g atoms/s\n" % ((atoms_per_second/number_of_bowties),
                                           atoms_per_second))
print("Total moles per second: %0.5g M/s" % moles_per_second)
print("Deposited volume: %0.5g cm^3/h" % hourly_print_volume)
print("Total beam current: %0.5g Amps" % total_beam_current)
print("Per-chamber current: %0.5g Amps" % per_chamber_beam_current)

#I find it infinitely amusing that "40 amps of aluminum" is "40 grams per second of aluminum" within 0.6%

Atoms per second: 
per chamber 6.1999e+15 atoms/s 
total 6.1999e+19 atoms/s

Total moles per second: 0.00010295 M/s
Deposited volume: 3.7052 cm^3/h
Total beam current: 9.9333 Amps
Per-chamber current: 0.00099333 Amps


## Neutral stuff.

Determine RMS neutral gas velocity at a certain temperature:

$$\sqrt{\frac{3RT}{m}}$$

In [599]:
bowtie_rms_neutral_velocity = math.sqrt((3*constants.Boltzmann*bowtie_temperature)/(print_species.mass*amu))
bowtie_rms_energy = 0.5*print_species.mass*amu*(bowtie_rms_neutral_velocity**2.0)
print("RMS particle velocity: %0.5g m/s" % bowtie_rms_neutral_velocity)
print("RMS particle energy: %0.5g eV" % (bowtie_rms_energy/constants.electron_volt))

RMS particle velocity: 1137.6 m/s
RMS particle energy: 0.18096 eV


Determine approximate neutral gas pressure in a virtual volume "1 thermal velocity" long

$$\text{PV}=\text{nRT}$$

(I'm not totally sure that you can do this.)

Also approximately determine mean free path with the neutral Van Der Waals radius.

In [609]:
R_constant = constants.Boltzmann*constants.Avogadro

#volume nozzle area times 1 thermal neutral injection velocity long
nozzle_virtual_volume = (math.pi*(bowtie_nozzle_radius**2))*bowtie_rms_neutral_velocity

bowtie_nozzle_pressure = ((moles_per_second/number_of_bowties)*R_constant*bowtie_temperature)/(nozzle_volume)

mean_free_path = 1.0/(((atoms_per_second/number_of_bowties)/nozzle_virtual_volume)*math.pi*((print_species.vdw_radius*1e-12)**2))


print("Bowtie nozzle pressure: %0.5g Pa" % bowtie_nozzle_pressure)
print("Approximate nozzle mean free path: %0.5g m" % mean_free_path)

#number density of atoms within that virtual volume - useful for DSMC stuff
print("nrho: %0.5g" % (atoms_per_second/nozzle_volume))

print("Knudsen number: local %0.5g, global %0.5g" % ((mean_free_path/bowtie_nozzle_radius),
                                                     (mean_free_path/build_platform_distance)))

print("Worst-case space charge density: %0.5g" % ((atoms_per_second/number_of_bowties)/nozzle_volume))

Bowtie nozzle pressure: 0.3079 Pa
Approx. species vapor pressure: 0.47493 Pa
Approximate nozzle mean free path: 0.59023 m
nrho: 1.5929e+23
Knudsen number: local 1788.6, global 19.674
Worst-case space charge density: 1.5929e+19


Now a rough approximation of the actual print species vapor pressure one can expect to find at that temperature:

(likely off by at least 50% - https://www.iap.tuwien.ac.at/www/surface/vapor_pressure is a more accurate source)

In [ ]:
vapor_pressure = Chemical(print_species.name).VaporPressure.calculate(bowtie_temperature,"BOILING_CRITICAL") # takes temp in K, returns pressure in P
print("Approx. species vapor pressure: %0.5g Pa" % vapor_pressure)

## Charged stuff.

(4/9(epsilon 0) (2*(electron charge)/(26 amu))^0.5 * ((50V)^(3/2))/(0.00005m)^2))*0.005 m^2

(4/9(epsilon 0) (2*(electron charge)/(electron mass))^0.5 * ((50V)^(3/2))/(0.00005m)^2))*0.005 m^2


In [601]:
acceleration_velocity = math.sqrt((2.0*acceleration_voltage*constants.electron_volt)/(print_species.mass*amu))
print("Accel. vel: %0.5g m/s" % acceleration_velocity)
print("Accel. power per chamber: %0.5g W, total %0.5g W " % (acceleration_voltage*per_chamber_beam_current
                                                        ,acceleration_voltage*total_beam_current))

beam_exit_velocity = bowtie_rms_neutral_velocity+acceleration_velocity

focus_field = util.scharge_efield(per_chamber_beam_current,beam_exit_velocity,ionization_chamber_radius)
print("Required focus field: %0.5g V/m" % focus_field)

#sine focus voltage equation would be nice
print("Required focus voltage: ~%0.5g V" % (focus_field*(ionization_chamber_radius/5.0)))


Accel. vel: 26743 m/s
Accel. power per chamber: 0.099333 W, total 993.33 W 
Required focus field: 4.2714e+05 V/m
Required focus voltage: ~128.14 V


In [602]:
per_chamber_ionization_power = ((atoms_per_second/number_of_bowties)*print_species.ionenergies[1]*constants.electron_volt)
total_ionization_power = per_chamber_ionization_power*number_of_bowties
print("Ionization power: per ~%0.5g W, total %0.5g W" % (per_chamber_ionization_power,total_ionization_power))

Ionization power: per ~0.0059458 W, total 59.458 W


 Charge assembly energy, circular approximation (eq. 8.7 FLP Vol II):
 TODO: add cylindrical eq. - this is horribly inaccurate, but at least it's an upper bound.

In [607]:
ionization_chamber_volume = (math.pi*(ionization_chamber_radius**2))*bowtie_rms_neutral_velocity


per_chamber_assembled_energy = (3.0/5.0)*(((atoms_per_second/number_of_bowties)* \
                                 (ionization_chamber_radius/bowtie_rms_neutral_velocity)*constants.e)**2.0) \
                                /(4.0*math.pi*constants.epsilon_0*ionization_chamber_radius)*(bowtie_rms_neutral_velocity/ionization_chamber_radius)

print("Charge assembly power: per ~%0.5g W, total %0.5g W" % (per_chamber_assembled_energy, per_chamber_assembled_energy*number_of_bowties))

Charge assembly power: per ~4.677 W, total 46770 W


Bowtie radiation power:

In [604]:
per_bowtie_power = (bowtie_hot_area * graphite_emissivity * constants.Stefan_Boltzmann * (bowtie_temperature**4.0))
total_bowtie_power = per_bowtie_power*number_of_bowties

print("Bowtie power: per %0.5g W, total %0.5g W" % (per_bowtie_power,total_bowtie_power))


Bowtie power: per 0.20694 W, total 2069.4 W


In [605]:
total_power_consumption = total_bowtie_power + (acceleration_voltage*total_beam_current) + per_chamber_ionization_power
print("Total power consumption: %0.5g W" % (total_power_consumption))

Total power consumption: 3062.7 W


ICP acceleration field, (line integral of Maxwell's second equation, eq. 4.5-3 of "Fundamentals"):

In [606]:
(-(1.0j*2.0*math.pi*ICP_ionization_frequency)/2.0)*

SyntaxError: invalid syntax (<ipython-input-606-2bfe023269aa>, line 1)